In [ ]:
from utils import spark_session, save_table

session = spark_session("purchase-suppression-dl-model")

In [ ]:
df_smp = session.table("bx_ps_samples_enriched_hr")
df_prch = session.table("bx_ps_user_prch_hist_hr")
df_vi = session.table("bx_ps_user_vi_hist_hr")

In [ ]:
from features import vi_label

config = dict()
config['sampling'] = {
    'ref_dt': '2024-05-01'
}

df_smp = vi_label(session, config, df_smp, df_vi)

In [ ]:
from pyspark.sql import functions as F

df_prch = (df_prch.groupBy("BUYER_ID").agg(F.to_json(
                                             F.collect_list(
                                                F.struct(*[F.col(c).alias(c) for c in df_prch.columns])
                                             )).alias("PURCHASE_HIST")))
df_vi = (df_vi.groupBy("BUYER_ID").agg(F.to_json(
                                             F.collect_list(
                                                F.struct(*[F.col(c).alias(c) for c in df_vi.columns])
                                             )).alias("VI_HIST")))
df = df_smp.join(df_prch, "BUYER_ID").join(df_vi, "BUYER_ID")

In [ ]:
# split data to train/validation/test sets
train, validation, test = df.randomSplit([0.7, 0.1, 0.2])

In [ ]:
train.write.option("maxRecordsPerFile", 200).mode("overwrite").parquet("hdfs://apollo-rno/user/b_perso/ps-dl/ps-dl-train-v1")
validation.write.option("maxRecordsPerFile", 200).mode("overwrite").parquet("hdfs://apollo-rno/user/b_perso/ps-dl/ps-dl-val-v1")
test.write.option("maxRecordsPerFile", 200).mode("overwrite").parquet("hdfs://apollo-rno/user/b_perso/ps-dl/ps-dl-test-v1")

In [ ]:
if session:
    session.stop()